---

# Tools for Computing Gradients and Basic Finite Element Matrices

The `gradient.hpp` file contains various tools for computing gradients and basic linear finite element matrices. Below are the key functionalities provided:

### Elemental Mass Matrix

The elemental mass matrix can be represented in two forms:

1. **Constant Scalar Factor:**
   $$ M = f \int_{K} \phi_i(x) \phi_j(x) \, dx $$
   Where $f$ is a scalar factor representing the constant value of a function over the finite element $K$.

2. **Linear Scalar Function:**
   $$ M = \int_{K} \phi_i(x) \phi_j(x) s(x) \, dx $$
   Here, $s(x)$ is a linear scalar function represented by its values at the element nodes:
   $$ s(x) = \sum_{k=1}^n s_k \phi_k(x) $$

### Elemental Stiffness Matrix

The elemental stiffness matrix is given by:
$$ K = f \int_{K} \nabla \phi_i(x) \nabla \phi_j(x) \, dx $$
Where $f$ again represents the constant value of a function over the finite element $K$.

### Additional Utilities

There are also utilities to compute the gradient of a function over a linear finite element.

### Documentation

All code is documented using Doxygen. You can produce the documentation by running Doxygen on the source code.

### Note

To avoid unnecessary reallocation of matrices, the functions for computing various element matrices may be transformed into methods of a class that stores the matrices as member variables. This improvement is not implemented here to keep the code as simple as possible but may be useful in the future.

--- 


In [ ]:
#include "LinearFiniteElement.hpp"
#include <fstream>
#include <iostream>
#include <sstream>
#include <vector>

int main() {
  // Read mesh from file
  std::string filename = "mesh.vtk";
  std::ifstream file(filename);
  if (!file) {
    std::cerr << "Failed to open file: " << filename << std::endl;
    return 1;
  }

  // Read element connectivity and nodes
  Eigen::Matrix<double, 3, Eigen::Dynamic> nodes;
  Eigen::Matrix<Eigen::Index, 4, Eigen::Dynamic> connectivity;
  int numElements;
  int numNodes;
  std::string line;
  std::istringstream ss;
  while (std::getline(file, line)) {
    if (line.find("CELLS") != std::string::npos) {
      // extract the substring starting from the first digit
      std::string numElementsStr =
          line.substr(line.find_first_of("0123456789"));
      // put the string in a stringstream
      ss.str(numElementsStr);
      ss.seekg(0, std::ios::beg);
      // read the number of elements
      ss >> numElements;
      connectivity.resize(4, numElements);
      for (int i = 0; i < numElements; i++) {
        int dummy;
        std::getline(file, line);
        ss.str(line);
        ss.seekg(0, std::ios::beg);
        ss >> dummy;
        for (int j = 0; j < 4; j++) {
          ss >> connectivity(j, i);
        }
      }
    } else if (line.find("POINTS") != std::string::npos) {
      std::string numNodesStr = line.substr(line.find_first_of("0123456789"));
      ss.str(numNodesStr);
      ss.seekg(0, std::ios::beg);
      ss >> numNodes;
      nodes.resize(3, numNodes);
      for (int i = 0; i < numNodes; i++) {
        std::getline(file, line);
        ss.str(line);
        ss.seekg(0, std::ios::beg);
        ss >> nodes(0, i) >> nodes(1, i) >> nodes(2, i);
      }
    }
  }
  // prepare global matrices
  Eigen::SparseMatrix<double> stiffnessMatrix(numNodes, numNodes);
  Eigen::SparseMatrix<double> massMatrix(numNodes, numNodes);
  std::vector<Eigen::Triplet<double>> triplets;
  triplets.reserve(numElements);
  for (int k = 0; k < numElements; ++k) {
    {
      for (auto i = 0; i < 4; ++i) {
        for (auto j = i; j < 4; ++j) {
          stiffnessMatrix.coeffRef(connectivity(i, k), connectivity(j, k)) = 0;
          stiffnessMatrix.coeffRef(connectivity(j, k), connectivity(i, k)) = 0;
          massMatrix.coeffRef(connectivity(i, k), connectivity(j, k)) = 0;
          massMatrix.coeffRef(connectivity(j, k), connectivity(i, k)) = 0;
        }
      }
    }
  }
  // Create a linear finite element
  apsc::LinearFiniteElement<3> linearFiniteElement;
  using Nodes = apsc::LinearFiniteElement<3>::Nodes;
  using Indexes = apsc::LinearFiniteElement<3>::Indexes;

  Nodes localNodes;
  Indexes globalNodeNumbers;
  for (auto k = 0; k < numElements; ++k) {
    // extract element data
    for (auto i = 0; i < 4; ++i) // node numbering
    {
      globalNodeNumbers(i) = connectivity(i, k);
      for (auto j = 0; j < 3; ++j) // local node coordinates
      {
        localNodes(j, i) = nodes(j, connectivity(i, k));
      }
    }
    linearFiniteElement.update(localNodes);
    linearFiniteElement.updateGlobalNodeNumbers(globalNodeNumbers);
    // Compute the local stiffness matrix
    linearFiniteElement.computeLocalStiffness();
    // Compute the local mass matrix
    linearFiniteElement.computeLocalMass();
    // Add the local matrices to the global matrices
    linearFiniteElement.updateGlobalStiffnessMatrix(stiffnessMatrix);
    linearFiniteElement.updateGlobalMassMatrix(massMatrix);
  }

  // Print stiffness matrix
  std::cout << "Stiffness Matrix:" << std::endl;
  std::cout << stiffnessMatrix << std::endl;

  // Print mass matrix
  std::cout << "Mass Matrix:" << std::endl;
  std::cout << massMatrix << std::endl;

  return 0;
}